In [1]:
import numpy as np
import pandas as pd
#import gensim
import matplotlib.pyplot as plt
import seaborn as sns
#from nltk.tokenize import word_tokenize
#from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pyarrow.parquet as pq

In [2]:
data2019=pd.read_pickle("data2019.pkl", compression='infer')

In [4]:
data2019

,quoteID,quotation,speaker,date,numOccurrences
1015,2019-09-26-057277,It's managing those zillions of microbial cell...,Thomas Crowther,2019-09-26 11:00:45,1
2314,2019-03-28-015294,Climate change is the single most important is...,Nicola Sturgeon,2019-03-28 14:55:43,1
2316,2019-06-28-012580,Climate change represents an existential threa...,Kamala Harris,2019-06-28 04:10:00,1
3028,2019-05-08-019902,"Even for Australians who can afford to pay, ge...",Dr Karl,2019-05-08 22:36:00,5
5577,2019-01-20-042295,The challenges this year will include global t...,Nguyen Xuan Phuc,2019-01-20 03:40:26,7
...,...,...,...,...,...
21759158,2019-06-06-003946,Air pollution is there. But it is not a questi...,Prakash Javadekar,2019-06-06 13:52:11,1
21760081,2019-07-10-012257,Climate change is a given.... Opening up new f...,Jan Haverkamp,2019-07-10 17:45:04,1
21760773,2019-07-15-018460,Given the critical juncture we are at with cli...,Alice Martin,2019-07-15 09:57:25,1
21762234,2019-06-05-040926,I think we had a great conversation and it was...,President Donald Trump,2019-06-05 14:00:16,1


In [5]:
QID = pd.read_csv('wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')
QID

,Label,Description
QID,,
Q31,Belgium,country in western Europe
Q45,Portugal,country in southwestern Europe
Q75,Internet,global system of connected computer networks
Q148,People's Republic of China,sovereign state in East Asia
Q155,Brazil,country in South America
...,...,...
Q106302506,didgeridooist,musician who plays the didgeridoo
Q106341153,biochemistry teacher,teacher of biochemistry at any level
Q106368830,2018 Wigan Metropolitan Borough Council electi...,NaN


In [6]:
speaker_df = pd.read_parquet("speaker_attributes.parquet", engine='pyarrow')
speaker_df.head()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None


In [14]:
len(speaker_df)

9055981

In [15]:
speaker_df=speaker_df.drop_duplicates(subset=['label'])

In [16]:
len(speaker_df)

7287299

In [129]:
n=1000
top=data2019['speaker'].value_counts()[:n].index.tolist()

In [130]:
sort = pd.Series(top, name = 'label').to_frame()
sort

,label
0,Jay Inslee
1,Bernie Sanders
2,Greta Thunberg
3,Alexandria Ocasio-Cortez
4,Elizabeth Warren
...,...
995,Olafur Eliasson
996,Piers Forster
997,Lisa Levin
998,Elaine Luria


In [131]:
#keep only top speakers
top_speakers=speaker_df[pd.DataFrame(speaker_df.label.tolist()).isin(top).any(1).values]


In [132]:
top_speakers_inorder = pd.merge(sort, top_speakers, how='left', on='label')


In [133]:
party=top_speakers_inorder.party
party = pd.Series(party, name = 'QID').to_frame()
party=party.explode(column='QID')

In [134]:
party = party.value_counts().rename_axis('QID').reset_index(name='counts')

In [139]:
party=pd.merge(party, QID, on='QID')
party


,QID,counts,Label_x,Description_x,Label_y,Description_y
0,Q29552,154,Democratic Party,political party in the United States,Democratic Party,political party in the United States
1,Q29468,40,Republican Party,major political party in the United States,Republican Party,major political party in the United States
2,Q9626,15,Conservative Party,political party in the United Kingdom,Conservative Party,political party in the United Kingdom
3,Q216082,13,Australian Labor Party,political party in Australia,Australian Labor Party,political party in Australia
4,Q327591,12,independent politician,individual not affiliated to any political party,independent politician,individual not affiliated to any political party
...,...,...,...,...,...,...
141,Q2660011,1,Québec solidaire,"political party in Quebec, Canada",Québec solidaire,"political party in Quebec, Canada"
142,Q239333,1,People's Party for Freedom and Democracy,Dutch political party,People's Party for Freedom and Democracy,Dutch political party
143,Q216517,1,Fianna Fáil,political party in the Republic of Ireland,Fianna Fáil,political party in the Republic of Ireland
144,Q21218940,1,Solidarity–People Before Profit,electoral alliance in Ireland,Solidarity–People Before Profit,electoral alliance in Ireland


In [17]:
QID.loc[QID.index.intersection(B_party)]

,Label,Description
QID,,
Q327591,independent politician,individual not affiliated to any political party
Q29552,Democratic Party,political party in the United States
Q6542163,Liberty Union Party,political party in Vermont
